In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

In [4]:
# Load the clustered data
df_labeled = pd.read_parquet('../../data/processed/clustered_data.parquet')

In [5]:
# Extract clusters and true labels
clusters = df_labeled['cluster'].to_numpy()
true_labels = df_labeled['incremento_teleassistenze'].to_numpy()

In [6]:

# Convert true labels to numeric values
label_encoder = LabelEncoder()
true_labels_numeric = label_encoder.fit_transform(true_labels)

In [7]:

# Reshape the arrays to be 2D
true_labels_numeric = true_labels_numeric.reshape(-1, 1)
clusters = clusters.reshape(-1, 1)

In [8]:
silhouette_vals = 0
mean_silhouette = 0

# Function to calculate and normalize Silhouette Score
def calculate_silhouette_score(true_labels_numeric, clusters):
    # Calculate the silhouette scores for each sample
    silhouette_vals = silhouette_samples(true_labels_numeric, clusters)

    # Calculate the mean silhouette score
    mean_silhouette = silhouette_score(true_labels_numeric, clusters)

    # Normalize the silhouette scores to a range between 0 and 1
    normalized_silhouette_vals = (silhouette_vals - silhouette_vals.min()) / (silhouette_vals.max() - silhouette_vals.min())
    normalized_mean_silhouette = (mean_silhouette - silhouette_vals.min()) / (silhouette_vals.max() - silhouette_vals.min())

    return normalized_mean_silhouette, normalized_silhouette_vals

In [9]:

# Calculate the normalized silhouette values
normalized_mean_silhouette, normalized_silhouette_vals = calculate_silhouette_score(true_labels_numeric, clusters)

/home/ignazio/ucbm/fia/Teleassistance-Supervised-Clustering/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:
# Print the results
print('Normalized Mean Silhouette Score:', normalized_mean_silhouette)
print('Normalized Silhouette Values:', normalized_silhouette_vals)

In [ ]:
# Plot the silhouette values for each sample
plt.figure(figsize=(10, 7))
y_lower = 10
for i in np.unique(clusters):
    ith_cluster_silhouette_vals = silhouette_vals[clusters == i]
    ith_cluster_silhouette_vals.sort()
    size_cluster_i = ith_cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i

    plt.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_vals, alpha=0.7)
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    y_lower = y_upper + 10

In [ ]:
plt.title('Silhouette plot for the various clusters')
plt.xlabel('Silhouette coefficient values')
plt.ylabel('Cluster label')
plt.axvline(x=mean_silhouette, color="red", linestyle="--")
plt.yticks([])
plt.show()